In [1]:
import docplex.mp.model as cpx
import pandas as pd
import numpy as np
import sys, os

### Data

In [2]:
options = pd.read_csv("data/filtered_tsla_options_w_greeks.csv")
options = options[options.date_current == "2020-07-13"]
options = options[options.days_to_expiry.isin([95.0, 186.0, 249.0])]

strikes = options.strike_price.value_counts()
strikes = strikes[strikes >= 3].index
options = options[options.strike_price.isin(strikes)]

In [3]:
options = options.iloc[:20]

### Model

In [4]:
## Params
N = len(options)
delta = np.concatenate((options.delta.values, -1 * options.delta.values))
gamma = np.concatenate((options.gamma.values, -1 * options.gamma.values))
vega = np.concatenate((options.vega.values, -1 * options.vega.values))
vomma = np.concatenate((options.vomma.values, -1 * options.vomma.values))

## Model
opt_model = cpx.Model(name="VolCrush Model")

## Variables
x_vars = {
    i : opt_model.binary_var(name=f"l{i}")
    for i in range(N)
}
x_vars.update({
    i+len(options) : opt_model.binary_var(name=f"s{i}")
    for i in range(N)
})

## Constraints
constraints = {
    i : opt_model.add_constraint(
        ct = opt_model.sum(x_vars[i] + x_vars[i + N]) <= 1,
        ctname = f"position_constraint_{i}"
    )
    for i in range(N)
}
constraints[len(constraints)] = opt_model.add_constraint(
    ct = opt_model.sum(x_vars[i] for i in range(2 * N)) <= 6,
    ctname = f"total_position_constraint"
)
constraints[len(constraints)] = opt_model.add_constraint(
    ct = opt_model.sum(x_vars[i] for i in range(2 * N)) >= 1,
    ctname = f"minimum_position_constraint"
)

## Objective
objective = opt_model.sum(
    (x_vars[i] * delta[i]) + (x_vars[i] * gamma[i]) + (x_vars[i] * vega[i]) + (x_vars[i] * vomma[i])
    for i in range(2 * N)
)
objective = opt_model.sum(
    
    opt_model.abs(
        opt_model.sum(
            x_vars[i] * delta[i]
            for i in range(2 * N)
        )
    )
    +
    opt_model.abs(
        opt_model.sum(
            x_vars[i] * gamma[i]
            for i in range(2 * N)
        )
    )
    +
    opt_model.abs(
        opt_model.sum(
            x_vars[i] * vega[i]
            for i in range(2 * N)
        )
    )
    +
    opt_model.sum(
        x_vars[i] * vomma[i]
        for i in range(2 * N)
    )
    
)
opt_model.minimize(objective)

In [7]:
opt_model.solve(log_output=True)

Version identifier: 12.10.0.0 | 2019-11-27 | 843d4de
CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              201903125

Root node processing (before b&c):
  Real time             =    0.00 sec. (0.00 ticks)
Parallel b&c, 8 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.06 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.00 sec. (0.00 ticks)


docplex.mp.solution.SolveSolution(obj=-0.008448,values={l11:1,l13:1,l14:..

### Results 

In [6]:
import pandas as pd
opt_df = pd.DataFrame.from_dict(x_vars, orient="index", columns = ["variable_object"])
opt_df.reset_index(inplace=True)
# CPLEX
opt_df["solution_value"] = opt_df["variable_object"].apply(lambda item: item.solution_value)
opt_df.drop(columns=["variable_object"], inplace=True)
opt_df.to_csv("optimization_solution.csv")

In [15]:
sol = opt_model.solution

In [17]:
sol.export_as_json_string()

'{"CPLEXSolution": {"version": "1.0", "header": {"problemName": "VolCrush Model", "objectiveValue": "-0.008447999999999647", "solved_by": "cplex_local"}, "variables": [{"index": "11", "name": "l11", "value": "1.0"}, {"index": "13", "name": "l13", "value": "1.0"}, {"index": "14", "name": "l14", "value": "1.0"}, {"index": "21", "name": "s1", "value": "1.0"}, {"index": "38", "name": "s18", "value": "1.0"}, {"index": "39", "name": "s19", "value": "1.0"}], "linearConstraints": [{"name": "position_constraint_0", "index": 0, "slack": 1.0}, {"name": "position_constraint_1", "index": 1}, {"name": "position_constraint_2", "index": 2, "slack": 1.0}, {"name": "position_constraint_3", "index": 3, "slack": 1.0}, {"name": "position_constraint_4", "index": 4, "slack": 1.0}, {"name": "position_constraint_5", "index": 5, "slack": 1.0}, {"name": "position_constraint_6", "index": 6, "slack": 1.0}, {"name": "position_constraint_7", "index": 7, "slack": 1.0}, {"name": "position_constraint_8", "index": 8, "s